In [1]:
# Import Library
import re
import pandas as pd
import ast
import numpy as np
from datetime import timedelta, datetime
import xlsxwriter
import joblib
from sklearn.model_selection import train_test_split

In [2]:
proportion = 3
if proportion == 1:
    test_size = 0.1
elif proportion == 2:
    test_size = 0.2
elif proportion == 3:
    test_size = 0.3
elif proportion == 4:
    test_size = 0.4
else:
    test_size = 0.2

In [3]:
df = pd.read_csv("./data/dataset_"+str(proportion)+"/cleaned_train.csv",sep=',')
df.tail()

,text,sentiment,cleaning,casefolding,tokenizing,normalisasi,stopword,steeming,cleaned
1045,nih twitter nya si bajingan homo tolol yang su...,negative,nih twitter nya si bajingan homo tolol yang su...,nih twitter nya si bajingan homo tolol yang su...,"['nih', 'twitter', 'nya', 'si', 'bajingan', 'h...","['nih', 'twitter', 'nya', 'sih', 'bajingan', '...","['nih', 'twitter', 'nya', 'sih', 'bajingan', '...","['nih', 'twitter', 'nya', 'sih', 'bajing', 'ho...",nih twitter nya sih bajing homo tolol suka jua...
1046,"resto ini cukup luas , suasana sangat menyenan...",positive,resto ini cukup luas suasana sangat menyenangk...,resto ini cukup luas suasana sangat menyenangk...,"['resto', 'ini', 'cukup', 'luas', 'suasana', '...","['resto', 'ini', 'cukup', 'luas', 'suasana', '...","['resto', 'luas', 'suasana', 'menyenangkan', '...","['resto', 'luas', 'suasana', 'senang', 'unjung...",resto luas suasana senang unjung pandang kota ...
1047,"jumat malam , bandung diguyur hujan namun alha...",positive,jumat malam bandung diguyur hujan namun alhamd...,jumat malam bandung diguyur hujan namun alhamd...,"['jumat', 'malam', 'bandung', 'diguyur', 'huja...","['jumat', 'malam', 'bandung', 'diguyur', 'huja...","['jumat', 'malam', 'bandung', 'diguyur', 'huja...","['jumat', 'malam', 'bandung', 'guyur', 'hujan'...",jumat malam bandung guyur hujan alhamdulillah ...
1048,tempat nya seperti rumah biasa namun sangat ny...,positive,tempat nya seperti rumah biasa namun sangat ny...,tempat nya seperti rumah biasa namun sangat ny...,"['tempat', 'nya', 'seperti', 'rumah', 'biasa',...","['tempat', 'nya', 'seperti', 'rumah', 'biasa',...","['nya', 'rumah', 'nyaman', 'udara', 'nya', 'se...","['nya', 'rumah', 'nyaman', 'udara', 'nya', 'se...",nya rumah nyaman udara nya sejuk terima kasih ...
1049,"kalau djarum memang agak tidak kuat , dia baru...",negative,kalau djarum memang agak tidak kuat dia baru k...,kalau djarum memang agak tidak kuat dia baru k...,"['kalau', 'djarum', 'memang', 'agak', 'tidak',...","['kalau', 'djarum', 'memang', 'agak', 'tidak',...","['djarum', 'kuat', 'kerasa', 'bagus', 'nya', '...","['djarum', 'kuat', 'rasa', 'bagus', 'nya', 'pa...",djarum kuat rasa bagus nya pas bakar


In [4]:
# Buat Fungsi Pembobotan (TFIDF)

def convert_document_to_list(documents):
    documents = ast.literal_eval(documents)
    return [document for document in documents]

def generate_df(documents):
    df_dict = {}
    _document = {}
    for document in documents:
        _document = dict.fromkeys(document)
        for term in _document:
            if term in df_dict:
                df_dict[term] += 1
            else:
                df_dict[term] = 1
    return df_dict

def calculate_df(document,document_frequency):
    df_dict = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    _document = dict.fromkeys(document)
    for term in _document:
        if term in document_frequency:
            df_dict[term] = document_frequency[term]
    return df_dict

def calculate_tf(document):
    tf_dict = {}
    # menghitung nilai tf pada tiap kata
    for term in document:
        if term in tf_dict:
            tf_dict[term] += 1
        else:
            tf_dict[term] = 1
    return tf_dict

def generate_idf(documents, df_term):
    idf_dict = {}
    total_document = len(documents)
    for document in documents:
        for term in document:
            idf_dict[term] = np.log((total_document+1) / (df_term[term]+1))+1
    return idf_dict

def calculate_idf(document, df_term, total_document):
    idf_dict = {}
    for term, df in document.items():  # document.items() untuk mengubah dict to list
        idf_dict[term] = np.log((total_document+1) / (df_term[term]+1))+1
    return idf_dict

def calculate_tfidf(document, idf):
    tfidf = {}
    for term, term_tf in document.items(): # document.items() untuk mengubah dict to list
        tfidf[term] = term_tf * idf[term]
    return tfidf

In [5]:
df["document_list"] = df["steeming"].apply(convert_document_to_list)
df.head()

,text,sentiment,cleaning,casefolding,tokenizing,normalisasi,stopword,steeming,cleaned,document_list
0,: menangkan bolt super 4 tidak lte amp ; merch...,neutral,menangkan bolt super tidak lte amp merchandis...,menangkan bolt super tidak lte amp merchandis...,"['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'lte', 'amp', '...","['menang', 'bolt', 'super', 'lte', 'amp', 'mer...",menang bolt super lte amp merchandise lucky do...,"[menang, bolt, super, lte, amp, merchandise, l..."
1,sudirman said jadi cagub jateng amp ; calon wa...,neutral,sudirman said jadi cagub jateng amp calon waki...,sudirman said jadi cagub jateng amp calon waki...,"['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...",sudirman said cagub jateng amp calon wakil gub...,"[sudirman, said, cagub, jateng, amp, calon, wa..."
2,"selamat sore ibu , perihal produk perbankan ba...",neutral,selamat sore ibu perihal produk perbankan bank...,selamat sore ibu perihal produk perbankan bank...,"['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'perihal', 'produk', 'perb...","['selamat', 'sore', 'perihal', 'produk', 'perb...",selamat sore perihal produk perban bank mandir...,"[selamat, sore, perihal, produk, perban, bank,..."
3,makan shabu shabu sambil ngobrol sama teman te...,positive,makan shabu shabu sambil ngobrol sama teman te...,makan shabu shabu sambil ngobrol sama teman te...,"['makan', 'shabu', 'shabu', 'sambil', 'ngobrol...","['makan', 'shabu', 'shabu', 'sambil', 'mengobr...","['makan', 'shabu', 'shabu', 'mengobrol', 'tema...","['makan', 'shabu', 'shabu', 'obrol', 'teman', ...",makan shabu shabu obrol teman teman nya asik m...,"[makan, shabu, shabu, obrol, teman, teman, nya..."
4,mj toko surabaya lantai 1 blok h3 no. 52 mengu...,neutral,mj toko surabaya lantai blok h no mengucapkan ...,mj toko surabaya lantai blok h no mengucapkan ...,"['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...",mj toko surabaya lantai blok h no selamat,"[mj, toko, surabaya, lantai, blok, h, no, sela..."


In [6]:
df["tf"] = df["document_list"].apply(calculate_tf)
df.head()

,text,sentiment,cleaning,casefolding,tokenizing,normalisasi,stopword,steeming,cleaned,document_list,tf
0,: menangkan bolt super 4 tidak lte amp ; merch...,neutral,menangkan bolt super tidak lte amp merchandis...,menangkan bolt super tidak lte amp merchandis...,"['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'lte', 'amp', '...","['menang', 'bolt', 'super', 'lte', 'amp', 'mer...",menang bolt super lte amp merchandise lucky do...,"[menang, bolt, super, lte, amp, merchandise, l...","{'menang': 1, 'bolt': 1, 'super': 1, 'lte': 1,..."
1,sudirman said jadi cagub jateng amp ; calon wa...,neutral,sudirman said jadi cagub jateng amp calon waki...,sudirman said jadi cagub jateng amp calon waki...,"['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...",sudirman said cagub jateng amp calon wakil gub...,"[sudirman, said, cagub, jateng, amp, calon, wa...","{'sudirman': 1, 'said': 1, 'cagub': 1, 'jateng..."
2,"selamat sore ibu , perihal produk perbankan ba...",neutral,selamat sore ibu perihal produk perbankan bank...,selamat sore ibu perihal produk perbankan bank...,"['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'perihal', 'produk', 'perb...","['selamat', 'sore', 'perihal', 'produk', 'perb...",selamat sore perihal produk perban bank mandir...,"[selamat, sore, perihal, produk, perban, bank,...","{'selamat': 1, 'sore': 1, 'perihal': 1, 'produ..."
3,makan shabu shabu sambil ngobrol sama teman te...,positive,makan shabu shabu sambil ngobrol sama teman te...,makan shabu shabu sambil ngobrol sama teman te...,"['makan', 'shabu', 'shabu', 'sambil', 'ngobrol...","['makan', 'shabu', 'shabu', 'sambil', 'mengobr...","['makan', 'shabu', 'shabu', 'mengobrol', 'tema...","['makan', 'shabu', 'shabu', 'obrol', 'teman', ...",makan shabu shabu obrol teman teman nya asik m...,"[makan, shabu, shabu, obrol, teman, teman, nya...","{'makan': 2, 'shabu': 2, 'obrol': 1, 'teman': ..."
4,mj toko surabaya lantai 1 blok h3 no. 52 mengu...,neutral,mj toko surabaya lantai blok h no mengucapkan ...,mj toko surabaya lantai blok h no mengucapkan ...,"['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...",mj toko surabaya lantai blok h no selamat,"[mj, toko, surabaya, lantai, blok, h, no, sela...","{'mj': 1, 'toko': 1, 'surabaya': 1, 'lantai': ..."


In [7]:
document_frequency = generate_df(df["document_list"])
# document_frequency.head()

In [8]:
df["df"] = df["document_list"].apply(lambda x: calculate_df(x, document_frequency))
df.head()

,text,sentiment,cleaning,casefolding,tokenizing,normalisasi,stopword,steeming,cleaned,document_list,tf,df
0,: menangkan bolt super 4 tidak lte amp ; merch...,neutral,menangkan bolt super tidak lte amp merchandis...,menangkan bolt super tidak lte amp merchandis...,"['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'lte', 'amp', '...","['menang', 'bolt', 'super', 'lte', 'amp', 'mer...",menang bolt super lte amp merchandise lucky do...,"[menang, bolt, super, lte, amp, merchandise, l...","{'menang': 1, 'bolt': 1, 'super': 1, 'lte': 1,...","{'menang': 12, 'bolt': 1, 'super': 7, 'lte': 1..."
1,sudirman said jadi cagub jateng amp ; calon wa...,neutral,sudirman said jadi cagub jateng amp calon waki...,sudirman said jadi cagub jateng amp calon waki...,"['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...",sudirman said cagub jateng amp calon wakil gub...,"[sudirman, said, cagub, jateng, amp, calon, wa...","{'sudirman': 1, 'said': 1, 'cagub': 1, 'jateng...","{'sudirman': 3, 'said': 2, 'cagub': 5, 'jateng..."
2,"selamat sore ibu , perihal produk perbankan ba...",neutral,selamat sore ibu perihal produk perbankan bank...,selamat sore ibu perihal produk perbankan bank...,"['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'perihal', 'produk', 'perb...","['selamat', 'sore', 'perihal', 'produk', 'perb...",selamat sore perihal produk perban bank mandir...,"[selamat, sore, perihal, produk, perban, bank,...","{'selamat': 1, 'sore': 1, 'perihal': 1, 'produ...","{'selamat': 10, 'sore': 14, 'perihal': 2, 'pro..."
3,makan shabu shabu sambil ngobrol sama teman te...,positive,makan shabu shabu sambil ngobrol sama teman te...,makan shabu shabu sambil ngobrol sama teman te...,"['makan', 'shabu', 'shabu', 'sambil', 'ngobrol...","['makan', 'shabu', 'shabu', 'sambil', 'mengobr...","['makan', 'shabu', 'shabu', 'mengobrol', 'tema...","['makan', 'shabu', 'shabu', 'obrol', 'teman', ...",makan shabu shabu obrol teman teman nya asik m...,"[makan, shabu, shabu, obrol, teman, teman, nya...","{'makan': 2, 'shabu': 2, 'obrol': 1, 'teman': ...","{'makan': 263, 'shabu': 2, 'obrol': 7, 'teman'..."
4,mj toko surabaya lantai 1 blok h3 no. 52 mengu...,neutral,mj toko surabaya lantai blok h no mengucapkan ...,mj toko surabaya lantai blok h no mengucapkan ...,"['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...",mj toko surabaya lantai blok h no selamat,"[mj, toko, surabaya, lantai, blok, h, no, sela...","{'mj': 1, 'toko': 1, 'surabaya': 1, 'lantai': ...","{'mj': 1, 'toko': 7, 'surabaya': 11, 'lantai':..."


In [9]:
n_document = len(df)
df["idf"] = df["df"].apply(lambda x: calculate_idf(x, document_frequency, n_document))
df.head()

,text,sentiment,cleaning,casefolding,tokenizing,normalisasi,stopword,steeming,cleaned,document_list,tf,df,idf
0,: menangkan bolt super 4 tidak lte amp ; merch...,neutral,menangkan bolt super tidak lte amp merchandis...,menangkan bolt super tidak lte amp merchandis...,"['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'lte', 'amp', '...","['menang', 'bolt', 'super', 'lte', 'amp', 'mer...",menang bolt super lte amp merchandise lucky do...,"[menang, bolt, super, lte, amp, merchandise, l...","{'menang': 1, 'bolt': 1, 'super': 1, 'lte': 1,...","{'menang': 12, 'bolt': 1, 'super': 7, 'lte': 1...","{'menang': 5.392548013415414, 'bolt': 7.264350..."
1,sudirman said jadi cagub jateng amp ; calon wa...,neutral,sudirman said jadi cagub jateng amp calon waki...,sudirman said jadi cagub jateng amp calon waki...,"['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...",sudirman said cagub jateng amp calon wakil gub...,"[sudirman, said, cagub, jateng, amp, calon, wa...","{'sudirman': 1, 'said': 1, 'cagub': 1, 'jateng...","{'sudirman': 3, 'said': 2, 'cagub': 5, 'jateng...","{'sudirman': 6.571203009757061, 'said': 6.8588..."
2,"selamat sore ibu , perihal produk perbankan ba...",neutral,selamat sore ibu perihal produk perbankan bank...,selamat sore ibu perihal produk perbankan bank...,"['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'perihal', 'produk', 'perb...","['selamat', 'sore', 'perihal', 'produk', 'perb...",selamat sore perihal produk perban bank mandir...,"[selamat, sore, perihal, produk, perban, bank,...","{'selamat': 1, 'sore': 1, 'perihal': 1, 'produ...","{'selamat': 10, 'sore': 14, 'perihal': 2, 'pro...","{'selamat': 5.559602098078581, 'sore': 5.24944..."
3,makan shabu shabu sambil ngobrol sama teman te...,positive,makan shabu shabu sambil ngobrol sama teman te...,makan shabu shabu sambil ngobrol sama teman te...,"['makan', 'shabu', 'shabu', 'sambil', 'ngobrol...","['makan', 'shabu', 'shabu', 'sambil', 'mengobr...","['makan', 'shabu', 'shabu', 'mengobrol', 'tema...","['makan', 'shabu', 'shabu', 'obrol', 'teman', ...",makan shabu shabu obrol teman teman nya asik m...,"[makan, shabu, shabu, obrol, teman, teman, nya...","{'makan': 2, 'shabu': 2, 'obrol': 1, 'teman': ...","{'makan': 263, 'shabu': 2, 'obrol': 7, 'teman'...","{'makan': 2.381548267730635, 'shabu': 6.858885..."
4,mj toko surabaya lantai 1 blok h3 no. 52 mengu...,neutral,mj toko surabaya lantai blok h no mengucapkan ...,mj toko surabaya lantai blok h no mengucapkan ...,"['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...",mj toko surabaya lantai blok h no selamat,"[mj, toko, surabaya, lantai, blok, h, no, sela...","{'mj': 1, 'toko': 1, 'surabaya': 1, 'lantai': ...","{'mj': 1, 'toko': 7, 'surabaya': 11, 'lantai':...","{'mj': 7.264350190317006, 'toko': 5.8780558291..."


In [10]:
idf = generate_idf(df["df"], document_frequency)

In [11]:
df["tfidf"] = df["tf"].apply(lambda x: calculate_tfidf(x, idf))
df.head()

,text,sentiment,cleaning,casefolding,tokenizing,normalisasi,stopword,steeming,cleaned,document_list,tf,df,idf,tfidf
0,: menangkan bolt super 4 tidak lte amp ; merch...,neutral,menangkan bolt super tidak lte amp merchandis...,menangkan bolt super tidak lte amp merchandis...,"['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'tidak', 'lte',...","['menangkan', 'bolt', 'super', 'lte', 'amp', '...","['menang', 'bolt', 'super', 'lte', 'amp', 'mer...",menang bolt super lte amp merchandise lucky do...,"[menang, bolt, super, lte, amp, merchandise, l...","{'menang': 1, 'bolt': 1, 'super': 1, 'lte': 1,...","{'menang': 12, 'bolt': 1, 'super': 7, 'lte': 1...","{'menang': 5.392548013415414, 'bolt': 7.264350...","{'menang': 5.392548013415414, 'bolt': 7.264350..."
1,sudirman said jadi cagub jateng amp ; calon wa...,neutral,sudirman said jadi cagub jateng amp calon waki...,sudirman said jadi cagub jateng amp calon waki...,"['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'jadi', 'cagub', 'jateng'...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...","['sudirman', 'said', 'cagub', 'jateng', 'amp',...",sudirman said cagub jateng amp calon wakil gub...,"[sudirman, said, cagub, jateng, amp, calon, wa...","{'sudirman': 1, 'said': 1, 'cagub': 1, 'jateng...","{'sudirman': 3, 'said': 2, 'cagub': 5, 'jateng...","{'sudirman': 6.571203009757061, 'said': 6.8588...","{'sudirman': 6.571203009757061, 'said': 6.8588..."
2,"selamat sore ibu , perihal produk perbankan ba...",neutral,selamat sore ibu perihal produk perbankan bank...,selamat sore ibu perihal produk perbankan bank...,"['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'ibu', 'perihal', 'produk'...","['selamat', 'sore', 'perihal', 'produk', 'perb...","['selamat', 'sore', 'perihal', 'produk', 'perb...",selamat sore perihal produk perban bank mandir...,"[selamat, sore, perihal, produk, perban, bank,...","{'selamat': 1, 'sore': 1, 'perihal': 1, 'produ...","{'selamat': 10, 'sore': 14, 'perihal': 2, 'pro...","{'selamat': 5.559602098078581, 'sore': 5.24944...","{'selamat': 5.559602098078581, 'sore': 5.24944..."
3,makan shabu shabu sambil ngobrol sama teman te...,positive,makan shabu shabu sambil ngobrol sama teman te...,makan shabu shabu sambil ngobrol sama teman te...,"['makan', 'shabu', 'shabu', 'sambil', 'ngobrol...","['makan', 'shabu', 'shabu', 'sambil', 'mengobr...","['makan', 'shabu', 'shabu', 'mengobrol', 'tema...","['makan', 'shabu', 'shabu', 'obrol', 'teman', ...",makan shabu shabu obrol teman teman nya asik m...,"[makan, shabu, shabu, obrol, teman, teman, nya...","{'makan': 2, 'shabu': 2, 'obrol': 1, 'teman': ...","{'makan': 263, 'shabu': 2, 'obrol': 7, 'teman'...","{'makan': 2.381548267730635, 'shabu': 6.858885...","{'makan': 4.76309653546127, 'shabu': 13.717770..."
4,mj toko surabaya lantai 1 blok h3 no. 52 mengu...,neutral,mj toko surabaya lantai blok h no mengucapkan ...,mj toko surabaya lantai blok h no mengucapkan ...,"['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...","['mj', 'toko', 'surabaya', 'lantai', 'blok', '...",mj toko surabaya lantai blok h no selamat,"[mj, toko, surabaya, lantai, blok, h, no, sela...","{'mj': 1, 'toko': 1, 'surabaya': 1, 'lantai': ...","{'mj': 1, 'toko': 7, 'surabaya': 11, 'lantai':...","{'mj': 7.264350190317006, 'toko': 5.8780558291...","{'mj': 7.264350190317006, 'toko': 5.8780558291..."


In [12]:
# Export to CSV
filename = "./data/dataset_"+str(proportion)+"/hasil/Hasil_Pembobotan_Tweet.csv"
df.to_csv(filename, index=False)

In [13]:
# Export to CSV Data term
df_terms = pd.DataFrame()
df_terms['term'] = document_frequency.keys()

list_tf = {}
for ind,val in df['tf'].items():
    for index,term_ in df_terms.iterrows():
        if term_['term'] in val.keys():
            list_tf[term_['term']] = val[term_['term']]
            
for ind,val in df['tf'].items():
    tf_ = {}
    for key,value in list_tf.items():
        if key in val.keys():
            tf_[key] = val[key]
        else:
            tf_[key] = 0
    df_terms['doc'+str(ind+1)] = tf_.values()
    
df_terms['df'] = document_frequency.values()
df_terms['idf'] = idf.values()
filename = "Hasil_Pembobotan_TF.csv"
df_terms.to_csv("./data/dataset_"+str(proportion)+"/hasil/"+filename, index=False)
df_terms.head()

/var/folders/12/mc7rk3yn1lz_7dgcx5dw2nt40000gn/T/ipykernel_15895/3850301898.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_terms['doc'+str(ind+1)] = tf_.values()
/var/folders/12/mc7rk3yn1lz_7dgcx5dw2nt40000gn/T/ipykernel_15895/3850301898.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_terms['df'] = document_frequency.values()
/var/folders/12/mc7rk3yn1lz_7dgcx5dw2nt40000gn/T/ipykernel_15895/3850301898.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

,term,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,...,doc1043,doc1044,doc1045,doc1046,doc1047,doc1048,doc1049,doc1050,df,idf
0,menang,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12,5.392548
1,bolt,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7.264350
2,super,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,7,5.878056
3,lte,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7.264350
4,amp,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,5.559602


In [14]:
df_terms_tfidf = df_terms.copy()

In [15]:
df_terms_tfidf['tfidf_positive'] = 0
df_terms_tfidf['tfidf_negative'] = 0
df_terms_tfidf['tfidf_neutral'] = 0
for ind,val in df.iterrows():
    df_terms_tfidf['doc'+str(ind+1)] = df_terms_tfidf['doc'+str(ind+1)]*df_terms_tfidf['idf']
    if val['sentiment'] == 'positive':
        df_terms_tfidf['tfidf_positive'] = df_terms_tfidf['tfidf_positive']+df_terms_tfidf['doc'+str(ind+1)]
    elif val['sentiment'] == 'negative':
        df_terms_tfidf['tfidf_negative'] = df_terms_tfidf['tfidf_negative']+df_terms_tfidf['doc'+str(ind+1)]
    else:
        df_terms_tfidf['tfidf_neutral'] = df_terms_tfidf['tfidf_neutral']+df_terms_tfidf['doc'+str(ind+1)]
filename = "Hasil_Pembobotan_Term_TFIDF.csv"
df_terms_tfidf.to_csv("./data/dataset_"+str(proportion)+"/hasil/"+filename, index=False)
df_terms_tfidf.head()

,term,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,...,doc1046,doc1047,doc1048,doc1049,doc1050,df,idf,tfidf_positive,tfidf_negative,tfidf_neutral
0,menang,5.392548,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12,5.392548,0.000000,21.570192,53.925480
1,bolt,7.264350,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,7.264350,0.000000,0.000000,7.264350
2,super,5.878056,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7,5.878056,35.268335,0.000000,5.878056
3,lte,7.264350,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,7.264350,0.000000,0.000000,7.264350
4,amp,5.559602,5.559602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,5.559602,0.000000,0.000000,61.155623


In [ ]:
with pd.ExcelWriter("./data/dataset_"+str(proportion)+"/hasil/Hasil_Pembobotan.xlsx", engine='xlsxwriter') as writer:    
    # Write each dataframe to a different worksheet.
    df.to_excel(writer, sheet_name='PreProcessing')
    df_terms.to_excel(writer, sheet_name='Perhitungan TF,DF,IDF')
    df_terms_tfidf.to_excel(writer, sheet_name='Perhitungan TFIDF')

In [ ]:
model_dump = {'train':df, 'tfidf': df_terms_tfidf, 'tf': df_terms}
joblib.dump(model_dump, open("./data/dataset_"+str(proportion)+"/pembobotan.model", "wb"))